In [20]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
from scipy.sparse import hstack
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Miguel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Miguel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
moviesML = pd.read_csv('../data/moviesEDA.csv')
vectorizer = TfidfVectorizer(stop_words='english')
lista_matrices = []
#movies_preprosesing = movies.head(5000)
for i in moviesML.columns:
    matriz = vectorizer.fit_transform(moviesML[i])
    lista_matrices.append(matriz)


In [22]:
moviesML

,genres,overview,production_companies,production_countries,spoken_languages,title,director
0,"Animation, Comedy, Family","Led by Woody, Andy's toys live happily in his ...",Pixar Animation Studios,United States of America,English,Toy Story,John Lasseter
1,"Adventure, Fantasy, Family",When siblings Judy and Peter discover an encha...,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,"English, Français",Jumanji,Joe Johnston
2,"Romance, Comedy",A family wedding reignites the ancient feud be...,"Warner Bros., Lancaster Gate",United States of America,English,Grumpier Old Men,Howard Deutch
3,"Comedy, Drama, Romance","Cheated on, mistreated and stepped on, the wom...",Twentieth Century Fox Film Corporation,United States of America,English,Waiting to Exhale,Forest Whitaker
4,Comedy,Just when George Banks has recovered from his ...,"Sandollar Productions, Touchstone Pictures",United States of America,English,Father of the Bride Part II,Charles Shyer
...,...,...,...,...,...,...,...
31222,"Horror, Mystery, Thriller",An unsuccessful sculptor saves a madman named ...,Universal Pictures,United States of America,English,House of Horrors,Georges Méliès
31223,Horror,A film archivist revisits the story of Rustin ...,"Neptune Salad Entertainment, Pirie Productions",United States of America,English,The Burkittsville 7,Georges Méliès
31224,Science Fiction,It's the year 3000 AD. The world's most danger...,Concorde-New Horizons,United States of America,English,Caged Heat 3000,Saul Blinkoff
31225,"Drama, Action, Romance","Yet another version of the classic epic, with ...","Westdeutscher Rundfunk (WDR), Working Title Fi...","Canada, Germany, United Kingdom, United States...",English,Robin Hood,Georges Méliès


In [23]:
combinacion_matrices = hstack(lista_matrices).tocsr() if len(lista_matrices) > 1 else lista_matrices[0]

In [24]:
def similitud_coseno(idx, matriz):
    return cosine_similarity(matriz[idx], matriz).flatten()

In [25]:
def recomendador(title:str):
    
    titulo_filmacion = moviesML[moviesML['title'].str.lower() == title.lower()]
    if titulo_filmacion.empty:
        return f'La película {title} no existe en la base de datos'

    idx = titulo_filmacion.index[0]
    score = similitud_coseno(idx, combinacion_matrices)

    lista_pelis = list(enumerate(score))
    lista_pelis = sorted(lista_pelis, key=lambda x: x[1], reverse=True)
    lista_pelis = lista_pelis[1:6]

    indices = [i[0] for i in lista_pelis]

    return moviesML['title'].iloc[indices].tolist() # type: ignore

In [28]:
recomendador("A Bug's Life")

['Toy Story', 'Toy Story 2', 'Tin Toy', 'Cars', 'Cars 2']

In [29]:
nltk.__version__

'3.8.1'